In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlb-pitch-data-20152018/pitches.csv
/kaggle/input/mlb-pitch-data-20152018/atbats.csv
/kaggle/input/mlb-pitch-data-20152018/games.csv
/kaggle/input/mlb-pitch-data-20152018/2019_games.csv
/kaggle/input/mlb-pitch-data-20152018/2019_atbats.csv
/kaggle/input/mlb-pitch-data-20152018/2019_pitches.csv
/kaggle/input/mlb-pitch-data-20152018/player_names.csv
/kaggle/input/mlb-pitch-data-20152018/ejections.csv


In [2]:
import seaborn as sns

In [3]:
data_raw = pd.read_csv('/kaggle/input/mlb-pitch-data-20152018/pitches.csv')
# Print out size, shape, and column names
print("size: " + str(data_raw.size))
print("shape: " + str(data_raw.shape))
print("columns: " + str(data_raw.columns))

size: 114686160
shape: (2867154, 40)
columns: Index(['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0',
       'z0', 'pfx_x', 'pfx_z', 'nasty', 'zone', 'code', 'type', 'pitch_type',
       'event_num', 'b_score', 'ab_id', 'b_count', 's_count', 'outs',
       'pitch_num', 'on_1b', 'on_2b', 'on_3b'],
      dtype='object')


In [4]:
df = data_raw
# dropping useless columns and rows with null values
df = df.drop(["nasty", "zone", "type_confidence", "pitch_type", "ab_id", "event_num"], axis = 1)
df = df.dropna()
# drop everything except swinging strikes and foul balls
df = df[df.code.isin(['S', 'F'])]
# only 2-strike counts
df = df[df.s_count == 2]
df = df[df.b_count == 3]
print(df.shape)

(55282, 34)


In [8]:
print(df.shape)
df = df.drop(["y0", "type", "b_score", "outs", "pitch_num", "b_count", "s_count", "on_1b", "on_2b", "on_3b"], axis = 1)

(55282, 24)


KeyError: "['y0' 'type' 'b_score' 'outs' 'pitch_num' 'b_count' 's_count' 'on_1b'\n 'on_2b' 'on_3b'] not found in axis"

In [12]:
print(df.columns)

Index(['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'z0', 'pfx_x', 'pfx_z',
       'code'],
      dtype='object')


In [9]:
# balance out the cases
strikes = df[df.code == 'S']
fouls = df[df.code == 'F']
fouls = fouls[0:len(strikes)]
tojoin = [strikes, fouls]
df = pd.concat(tojoin)
print(strikes.shape)
df.shape

(15199, 24)


(30398, 24)

In [19]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import sklearn.model_selection as tts

In [44]:
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
code_encoded = le.fit_transform(df.code) #S = 1, F = 0
print(code_encoded)

[1 1 1 ... 0 0 0]


In [45]:
features_train, features_test, labels_train, labels_test = tts.train_test_split(df.transpose()[:23].transpose(), code_encoded, test_size = 0.3, random_state = 69)

In [46]:
print(features_train.shape)
print(features_test.shape)
print(labels_train.shape)
print(labels_test.shape)

(21278, 23)
(9120, 23)
(21278,)
(9120,)


In [47]:
print(labels_test)
print(type(labels_test))

[1 1 0 ... 0 1 1]
<class 'numpy.ndarray'>


In [48]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(features_train, labels_train)

GaussianNB()

In [39]:
labels_train = labels_train.tolist()

In [49]:
predictions = model.predict(features_test)
print(predictions)

[1 1 0 ... 1 0 0]


In [54]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?

print("Accuracy:",metrics.accuracy_score(labels_test, predictions))

# right = 0
# for i in range(len(predictions)):
#     if (predictions[i] == labels_train[i]):
#         right += 1
        
# print(right / len(predictions))

Accuracy: 0.6115131578947368


In [ ]:
from sklearn.neural_network import MLPRegressor
fear = MLPRegressor(random_state=1, max_iter=100).fit(features_train, labels_train)
fear.score(features_test, labels_test)

In [ ]:
degree = 5
polyreg = make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(features_train,labels_train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.scatter(features_train,labels_train)
plt.plot(features_test,polyreg.predict(features_test),color="black")
plt.title("Polynomial regression with degree "+str(degree))
plt.show()

In [ ]:
# convert pitch outcomes to numbers so they can be plotted
# higher number = better outcome
# outcomes = {'S': 1, 'C': 1, 'W': 1, 'T': 0.75, 'F': 0.5, 'X': 0.25, 'D': 0, 'E': 0 }
# df['code_num'] = df['code'].map(outcomes)
# df = df.drop(['code'], axis = 1) 
sns.displot(df.code_num)